In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/데이터_전처리_파일_선박명.csv', encoding='cp949')
df.head(2)

,Unnamed: 0,선박명,청구서번호,No.,Subject,Machinery,Assembly,청구품목,Part No.1,Part No.2,...,창고입고수량,Control No.,입고창고,창고출고,창고출고수량,출고선박,출고운반선,선박입고,선박입고수량,완료 여부
0,0,ELS,ELS-BS-ESP-2004002,12,NO.1 A/E C18 MAJOR O/H PARTS,NO.1 GENERATOR ENGINE,323-6480 LINES GP-FUEL,SEAL-O-RING-STOR,7.00E-275,0,...,1,혜인 발주,BS,2020-09-19,1,BLO,본선 선적,0,0,0
1,1,OCA,OCA-BS-ESP-2010004,5,운전시간 대비 #2 AUX ENG TOP END O/H PARTS,NO.2 GENERATOR ENGINE,GASKET KIT,OIL COOLER & LINES,7.00E-275,0,...,1,혜인 발주,BS,2020-09-18,1,JBG,본선 선적,0,0,0


In [ ]:
# 대분류, 중분류, 소분류 데이터 전처리
data = list(df['Machinery']  + ' ' + df['Assembly'] + ' ' + df['청구품목'] )

In [ ]:
data[0]

'NO.1 GENERATOR ENGINE 323-6480 LINES GP-FUEL  SEAL-O-RING-STOR'

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
list(df['key2'].unique())

['COOLER',
 'PUMP',
 'BEARING',
 'GAUGE',
 'SWITCH',
 'VALVE',
 'GASKET',
 'O-RING',
 'CYLINDER',
 'PIN',
 'CONNECTING ROD',
 'PISTON',
 'GEAR',
 'KEY',
 'SPACER',
 'TOOL',
 'WASHER',
 'COVER',
 'ANODES',
 'SHAFT',
 'BUSHING',
 'CONTROL',
 'SPRING',
 'NUT',
 'PIPE',
 'FILTER',
 'BOLT',
 'CLAMP',
 'SCREW',
 'LINK',
 'DAMPER',
 'HOSE',
 'BRAKE',
 'TUBE',
 'PLATE',
 'MOTOR',
 'WINCH',
 'DISC',
 'ADAPTER',
 'BLOCK',
 'CARTRIDGE',
 'INJECTOR',
 'UNIT',
 'MANIFOLD',
 'LIQUID',
 'GENERATOR',
 'ARM',
 'HARNESS',
 'SEAT',
 'BELT',
 'CLIP',
 'LINE',
 'WIRE',
 'TRANSFORMER',
 'MONITOR',
 'NET',
 'BINO',
 'BATTERY',
 'TURBINE',
 'COMPRESSOR',
 'CARD']

대분류, 중분류, 소분류를 feature 값으로 사용하고, target 값으로는 COOLER, GASKET, GEAR, O-RING 등 61개 분류명을 사용하여 다중 분류 모델을 만들 수 있습니다.

전처리 과정에서는 대분류, 중분류, 소분류를 텍스트로 받아와서 토큰화하여 리스트 형태로 만들고, target 값을 숫자로 인코딩하여 넘파이 배열 형태로 만들어줍니다. 그 후, 해당 데이터를 분류 모델에 학습시켜 예측 모델을 만들어서 새로운 입력 데이터에 대해 분류할 수 있습니다.

예를 들어, 다음과 같은 코드로 분류 모델을 만들 수 있습니다. (사용하는 머신러닝 라이브러리와 분류 알고리즘에 따라 코드는 달라질 수 있습니다.)

 CountVectorizer를 사용하여 텍스트 데이터를 feature 벡터로 변환하고, LabelEncoder를 이용하여 target 값을 숫자로 인코딩하였습니다. 이후 train_test_split 함수를 사용하여 데이터를 train, test 데이터로 분리하고, OneVsRestClassifier를 사용하여 SVM 분류기를 만들어 분류 모델을 학습시켰습니다. 마지막으로, 새로운 입력 데이터를 예측하여 target 값을 출력하였습니다.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC


In [ ]:

# 대분류, 중분류, 소분류 텍스트 데이터
data = list(df['Machinery']  + ' ' + df['Assembly'] + ' ' + df['청구품목'] )


In [ ]:
# target 값
target =list(df['key2'])


In [ ]:
# CountVectorizer를 이용하여 feature 벡터 생성
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data)

In [ ]:
# LabelEncoder를 이용하여 target 값을 숫자로 인코딩
le = LabelEncoder()
y = le.fit_transform(target)

In [ ]:
# train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# 분류 알고리즘으로 SVM 사용
clf = OneVsRestClassifier(SVC(kernel='linear', probability=True))
clf.fit(X_train, y_train)

KeyboardInterrupt: ignored

In [ ]:
new_data = data
X_new = vectorizer.transform(new_data)
y_new = clf.predict(X_new)
predicted_target = le.inverse_transform(y_new)
print(predicted_target)

In [ ]:
pcheck = pd.DataFrame()
pcheck['data'] =data
pcheck['key2'] = df['key2']
pcheck['predict'] = predicted_target
pcheck['check'] = pcheck['key2'] == pcheck['predict']
pcheck

In [ ]:
pcheck[pcheck['check']==False]

In [ ]:
pcheck['check'] = pcheck['key2'] == pcheck['predict']
pcheck

In [ ]:
sum(pcheck['check']) / len(pcheck['check'])

In [ ]:
y_new = clf.predict(X_test)
predicted_target = le.inverse_transform(y_new)
print(predicted_target)

In [ ]:
predicted = list(le.inverse_transform(y_test))

In [ ]:
pcheck = pd.DataFrame()
pcheck['key2'] = predicted
pcheck['predict'] = predicted_target

pcheck

In [ ]:
pcheck['check'] = pcheck['key2'] == pcheck['predict']
sum(pcheck['check']) / len(pcheck['check'])

In [ ]:
from sklearn.metrics import accuracy_score # 정확도 함수

In [ ]:
# 분류 알고리즘으로 SVM 사용
clf = OneVsRestClassifier(SVC(kernel='linear', probability=True))
clf.fit(X_train, y_train)

predict1 = clf.predict(X_train)
print(accuracy_score(y_train,predict1))

0.8485346981051606


In [ ]:
# 분류 알고리즘으로 LogisticRegression 사용
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)

predict1 = clf.predict(X_train)
print(accuracy_score(y_train,predict1))

0.8406750746359593


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=200, max_depth=20,random_state=0)
clf.fit(X_train,y_train)

predict1 = clf.predict(X_train)
print(accuracy_score(y_train,predict1))

0.6308414062024006


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=200, max_depth=50,random_state=0)
clf.fit(X_train,y_train)

predict1 = clf.predict(X_train)
print(accuracy_score(y_train,predict1))

0.8644367269847072


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=200, max_depth=100,random_state=0)
clf.fit(X_train,y_train)

predict1 = clf.predict(X_train)
print(accuracy_score(y_train,predict1))

0.9004447693901176


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=200, max_depth=200,random_state=0)
clf.fit(X_train,y_train)

predict1 = clf.predict(X_train)
print(accuracy_score(y_train,predict1))

0.9007494059586912


In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)

predict1 = clf.predict(X_train)
print(accuracy_score(y_train,predict1))

0.9007494059586912


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
params = { 
    'n_estimators':[100,50],
     'learning_rate':[0.05,0.1]
         }



* SVC(kernel="linear", probability=True))

> RandomForestClassifier(n_estimators=200, max_depth=200,random_state=0)

> DecisionTreeClassifier()

* KNeighborsClassifier(n_neighbors = 3)
* GradientBoostingClassifier(random_state=0)
* SGDClassifier(max_iter=10000)

* AdaBoostClassifier(n_estimators=30, random_state=10, learning_rate=0.1



In [ ]:
lists = [OneVsRestClassifier(SVC(kernel="linear", probability=True)), RandomForestClassifier(n_estimators=200, max_depth=200,random_state=0), DecisionTreeClassifier(), KNeighborsClassifier(n_neighbors = 3), GradientBoostingClassifier(random_state=0), SGDClassifier(max_iter=10000), AdaBoostClassifier(n_estimators=30, random_state=10, learning_rate=0.1)]

for i in range(0, len(lists)) :
  clf = lists[i]
  clf.fit(X_train,y_train)

  predict1 = clf.predict(X_train)
  print(accuracy_score(y_train,predict1))

0.8485346981051606
0.9007494059586912
0.9007494059586912
0.787424602449278
0.8248949003838421
0.8420764028513983
0.19880582465119112


In [ ]:
clf = RandomForestClassifier(n_estimators=250, max_depth=200,random_state=0)
clf.fit(X_train,y_train)

predict1 = clf.predict(X_train)
print(accuracy_score(y_train,predict1))

0.9007494059586912


In [ ]:
clf = RandomForestClassifier(n_estimators=250, max_depth=250,random_state=0)
clf.fit(X_train,y_train)

predict1 = clf.predict(X_train)
print(accuracy_score(y_train,predict1))

0.9007494059586912


In [ ]:
for i in range(1, 10) : 
  clf = RandomForestClassifier(n_estimators=250+i*10, max_depth=250+i*10,random_state=i)
  clf.fit(X_train,y_train)

  predict1 = clf.predict(X_train)
  print(i, accuracy_score(y_train,predict1))

1 0.9007494059586912
2 0.9007494059586912
3 0.9007494059586912
4 0.9007494059586912
5 0.9007494059586912
6 0.9007494059586912
7 0.9007494059586912
8 0.9007494059586912
9 0.9007494059586912
